In [1]:
import pyltr
import numpy as np
import sklearn.tree
import random
import math

In [2]:
def randomize_data(data):
    sessions_length = 20
    for sessions in range(0, len(data)/20):
        session_data = data[sessions*20:(sessions + 1)* 20,:]
        np.random.shuffle(session_data)
        data[sessions*20:(sessions + 1)* 20,:] = session_data
    return data 
data_archive = np.load('data/train_suggest_matrix.npz')
data = data_archive['arr_0']
data = randomize_data(data)

num_sessions = len(data)/20
test = 0.15
train = 0.85



train_idx = int((num_sessions * 0.85) + 0.5)

train_data =data[0:train_idx*20,:]
val_data =data[train_idx*20:,:]

In [3]:


#col0=anchor ID, col1=suggestion ID. col 2-19 features, col 19 is scores
def make_sessions_ids(anchor_ids):
    
    sessions_ids = np.zeros([len(anchor_ids)])
    iters = len(anchor_ids) /20
    for i in range(iters):
        sessions_ids[i*20:(i+1)*20] = i
    return sessions_ids


train_data_anchor_ids = train_data[:,0]
train_data_suggestion_ids = train_data[:,1]
train_data_features = train_data[:,2:18]
train_data_label_scores = train_data[:,19]
train_data_session_ids = make_sessions_ids(train_data_anchor_ids)



val_data_anchor_ids = val_data[:,0]
val_data_suggestion_ids = val_data[:,1]
val_data_features = val_data[:,2:18]
val_data_label_scores = val_data[:,19]
val_data_session_ids = make_sessions_ids(val_data_anchor_ids)

413640
73000


In [4]:
metric = pyltr.metrics.NDCG(k=20)

In [ ]:
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=5,
    learning_rate=0.02,
    max_features=0.3,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=4,
    verbose=1,
    random_state=None
)           

monitor = pyltr.models.monitors.ValidationMonitor(
    val_data_features, val_data_label_scores, val_data_session_ids, metric=metric, stop_after=5)

In [ ]:
model.fit(train_data_features, train_data_label_scores, train_data_session_ids)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       1.0000       0.7723       48.88s                                         
    2       1.0000       0.0000       36.91s                                         
    3       1.0000       0.0000       24.18s                                         
    4       1.0000       0.0000       11.93s                                         


In [ ]:
Epred = model.predict(val_data_features)

print 'Random ranking:', metric.calc_mean_random(val_data_session_ids, val_data_label_scores)
print 'Our model:', metric.calc_mean(val_data_session_ids, val_data_label_scores, Epred)

In [ ]:
print np.argmax(val_data_label_scores[0:20])
print np.argmax(Epred[0:20])
print np.argmax(val_data_label_scores[20:40])
print np.argmax(Epred[20:40])


